# Scikit-learn packages

- sklearn套件是專門針對機器學習設計的套件
- 功能包含資料前處理、建立各類模型、計算評估指標
- sklearn主要接受的資料格式為numpy.ndarray，但由於pandas資料表格式也與numpy相容因此可通用

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

這次我們嘗試著整理比較複雜的資料，除了直接預測模型之外也加入一些前處理的步驟

## Load Data

https://www.kaggle.com/c/titanic/data

In [ ]:
train_dat = pd.read_csv('titanic/train.csv')
test_dat = pd.read_csv('titanic/test.csv')

print(train_dat.shape)
print(test_dat.shape)

In [ ]:
train_dat.head()

In [ ]:
train_dat.dtypes

In [ ]:
full_dat = pd.concat([train_dat, test_dat], sort = False)
full_dat.reset_index(drop = True, inplace = True)

full_dat.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1, inplace = True)

## Preprocess
- missing imputation
- one-hot / label encoding
- normalization

### missing imputation

In [ ]:
full_dat.isna().sum()

In [ ]:
# missing imputation

full_dat['Age'].fillna(full_dat['Age'].median(), inplace = True)
full_dat['Embarked'].fillna(full_dat['Embarked'].mode()[0], inplace = True)
full_dat['Fare'].fillna(full_dat['Fare'].median(), inplace = True)

In [ ]:
full_dat.isna().sum()

### one-hot encoding

In [ ]:
# one-hot encoding

one_hot_dat = pd.get_dummies(full_dat, columns = ['Pclass','Sex','Embarked'])
one_hot_dat.head()

In [ ]:
# from sklearn.preprocessing import OneHotEncoder (個人覺得不好用)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

fdat_cp = full_dat.copy()

for col in ['Pclass','Sex','Embarked']:
    fdat_cp[col] = LabelEncoder().fit_transform(fdat_cp[col])


In [ ]:
one_hot_columns = OneHotEncoder(sparse = False).fit_transform(fdat_cp[['Pclass','Sex','Embarked']])

one_hot_columns[:5,:]

In [ ]:
fdat_cp[['Pclass','Sex','Embarked']].head()

### normalization

In [ ]:
# normalization
from sklearn.preprocessing import StandardScaler

std_s = StandardScaler()

survived_ = one_hot_dat['Survived']
one_hot_dat.drop('Survived', axis = 1, inplace = True)

normalize_dat = std_s.fit_transform(one_hot_dat)

In [ ]:
print(type(normalize_dat))
print(normalize_dat.shape)

In [ ]:
normalize_dat.sum(axis = 0)

## Train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

test_index = survived_.isna()

train_x = normalize_dat[~test_index]
test_x = normalize_dat[test_index]
train_y = survived_[~test_index]

In [ ]:
t_x, v_x, t_y, v_y = train_test_split(train_x, train_y, test_size = 0.2, shuffle = True, random_state = 412)

print(t_x.shape)
print(v_x.shape)

## Build model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_model.fit(t_x, t_y)

In [ ]:
print('training score (decision tree : {:.3f}'.format(dt_model.score(t_x, t_y)))
print('validation score (decision tree : {:.3f}'.format(dt_model.score(v_x, v_y)))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true = v_y, y_pred = dt_model.predict(v_x))

In [ ]:
test_prediction = dt_model.predict(test_x)
test_dat['Survived'] = test_prediction

In [ ]:
test_dat.head()

In [ ]:
test_dat.to_csv('titanic_prediction.csv', index = False)

---

## 練習時間

請使用不同模型預測titanic資料集中的乘客存活與否，並看一下結果是否比決策樹還要來得好？

In [ ]:
# your code starts from here

---

## 監督式學習2.0

做完以上的範例與練習，希望大家能夠

- 進行基本的資料前處理動作
- 能夠比較不同的模型效果